# Exercise *04*: A/B-testing

## Create a connection to the database using the library *sqlite3*:

In [1]:
import sqlite3

from sqlite3 import Connection

In [2]:
db_path: str = "../../data/checking_logs.sqlite"

In [3]:
conn: Connection = sqlite3.connect(db_path)

## Using only one query for each of the groups, create two dataframes: `test_results` and `control_results` with the columns `time` and `avg_diff` and only two rows:

* *time* should have the values: after and before:
* *avg_diff* contains the average delta among all the users for the time period before each of them made their first visit to the page and afterward:
* Only take into account the users that have observations before and after:
* We still are not using the lab *’project1’*.

In [4]:
import pandas as pd

from pandas import DataFrame 

In [5]:
sql_test_query: str = """
                      SELECT
                          CASE
                              WHEN test.first_commit_ts < test.first_view_ts THEN 'before'
                              ELSE 'after'
                          END AS time,
                          AVG((unixepoch(test.first_commit_ts) - deadlines.deadlines) / 3600) AS avg_diff
                      FROM
                          test
                      INNER JOIN
                          deadlines ON test.labname = deadlines.labs
                      WHERE
                          test.labname != 'project1' AND
                          test.first_commit_ts IS NOT NULL AND
                          test.first_view_ts IS NOT NULL
                      GROUP BY
                          time;
                      """

sql_control_query: str = """
                         SELECT
                             CASE
                                 WHEN control.first_commit_ts < control.first_view_ts THEN 'before'
                                 ELSE 'after'
                             END AS time,
                             AVG((unixepoch(control.first_commit_ts) - deadlines.deadlines) / 3600) AS avg_diff
                         FROM
                             control
                         INNER JOIN
                             deadlines ON control.labname = deadlines.labs
                         WHERE
                             control.labname != 'project1' AND
                             control.first_commit_ts IS NOT NULL AND
                             control.first_view_ts IS NOT NULL
                         GROUP BY
                             time;
                         """

In [6]:
test_results: DataFrame = pd.io.sql.read_sql(
    sql_test_query,
    conn
)

control_results: DataFrame = pd.io.sql.read_sql(
    sql_control_query,
    conn
)

In [7]:
test_results

,time,avg_diff
0,after,-103.40625
1,before,-60.56250


In [8]:
control_results

,time,avg_diff
0,after,-112.710526
1,before,-99.464286


## Close the connection:

In [9]:
conn.close()

## Have the answer: *Did the hypothesis turn out to be true and the page does affect the students behavior*?

In [10]:
print("Yes" if (test_results.iloc[:, 1][0] < test_results.iloc[:, 1][1]) and (control_results.iloc[:, 1][0] < control_results.iloc[:, 1][1]) else "No")

Yes
